# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [1]:
## SETUP used later
import sys
import os
import json
import pandas as pd
sys.path.insert(1, '/locale/data/jupyter/prando/notebook/sparqlthesis/')
import modules.evaluation as evaluation
from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-history5-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://gracevirtuoso.dei.unipd.it/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://gracevirtuoso.dei.unipd.it/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# History Workflow Series ("Euro explorative search") 

Consider the following exploratory information need:

> investigate the Euro currency, which are the countries that currently use the Euro and which were their currencies before the Euro. Take a look also to other currencies and find which are the most used.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | occupation    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q38`| Italy   | node |





Also consider

```
wd:Q38 ?p ?obj .
```

is the BGP to retrieve all **properties of Italy**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.



The workload should:


1. Identify the BGP for currency

2. Return the countries which currently use Euro

3. Return the countries belonging to the European Union, with a currency different from the Euro (return also their currency).

4. For each country that now uses Euro return which was the currencies used before

5. Return all types of dollar in the world (US Dollar, Australian Dollar, ecc...)

6. Return the currencies used in at least 2 different countries (return also the number of countries using each currency).

7. Consider the currencies obtained in task 6, for each currency return the number of people using that currency (hint: consider the maximum popolation available for each country).

In [2]:
## startup the evaluation
# setup the file and create the empty json
ipname = "h5.ipynb"
pt = os.getcwd()+os.sep+ipname
evaluation.setup(pt)

The index of this workflow is: 1_1


## Task 1

In [3]:
# find properties of Italy
queryString = """
SELECT DISTINCT ?p ?name
WHERE { 
    wd:Q38 ?p ?obj.
    ?p sc:name ?name.
} 
order by ?name
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P8061'), ('name', 'AGROVOC ID')]
[('p', 'http://www.wikidata.org/prop/direct/P5198'), ('name', 'ASC Leiden Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P6150'), ('name', 'Academy Awards Database nominee ID')]
[('p', 'http://www.wikidata.org/prop/direct/P8895'), ('name', 'All the Tropes identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P7870'), ('name', 'Analysis & Policy Observatory term ID')]
[('p', 'http://www.wikidata.org/prop/direct/P8785'), ('name', 'AniDB tag ID')]
[('p', 'http://www.wikidata.org/prop/direct/P9629'), ('name', 'Armeniapedia ID')]
[('p', 'http://www.wikidata.org/prop/direct/P6200'), ('name', 'BBC News topic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1617'), ('name', 'BBC Things ID')]
[('p', 'http://www.wikidata.org/prop/direct/P9037'), ('name', 'BHCL UUID')]
[('p', 'http://www.wikidata.org/prop/direct/P2581'), ('name', 'BabelNet ID')]
[('p', 'http://www.wikidata.org/prop/direct/P7832

In [4]:
# use the property currency
queryString = """
SELECT DISTINCT ?obj ?name
WHERE { 
    wd:Q38 wdt:P38 ?obj.
    ?obj sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q4916'), ('name', 'euro')]
1


In [5]:
# instance of
queryString = """
SELECT DISTINCT ?obj ?name
WHERE { 
    wd:Q4916 wdt:P31 ?obj.
    ?obj sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q8142'), ('name', 'currency')]
[('obj', 'http://www.wikidata.org/entity/Q28795800'), ('name', 'decimal currency')]
2


In [6]:
### insert the result of TASK 1 in the file
og_uri = "http://www.wikidata.org/entity/Q8142"
og_name = "currency"
obj = {"uri":og_uri,"name":og_name}
evaluation.add_result(evaluation.get_index_workflow(pt),"1", evaluation.TYPE_SINGLE ,"uri", [obj] ,"all")

The index of this workflow is: 1_1
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow1_1.json
JSON object updated


`wd:Q8142` is the class of currency

## Task 2

In [8]:
# properties of euro
queryString = """
SELECT DISTINCT ?prop ?name
WHERE { 
    wd:Q4916 ?prop ?obj.
    ?prop sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P1001'), ('name', 'applies to jurisdiction')]
[('prop', 'http://www.wikidata.org/prop/direct/P1245'), ('name', 'OmegaWiki Defined Meaning')]
[('prop', 'http://www.wikidata.org/prop/direct/P1282'), ('name', 'OpenStreetMap tag or key')]
[('prop', 'http://www.wikidata.org/prop/direct/P1365'), ('name', 'replaces')]
[('prop', 'http://www.wikidata.org/prop/direct/P138'), ('name', 'named after')]
[('prop', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P1482'), ('name', 'Stack Exchange tag')]
[('prop', 'http://www.wikidata.org/prop/direct/P154'), ('name', 'logo image')]
[('prop', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received')]
[('prop', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('prop', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('prop', 'http://www.wikidata.org/prop/direct/P181

In [9]:
# use country
queryString = """
SELECT DISTINCT ?obj ?name
WHERE { 
    wd:Q4916 wdt:P17 ?obj.
    ?obj sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q142'), ('name', 'France')]
[('obj', 'http://www.wikidata.org/entity/Q183'), ('name', 'Germany')]
[('obj', 'http://www.wikidata.org/entity/Q191'), ('name', 'Estonia')]
[('obj', 'http://www.wikidata.org/entity/Q211'), ('name', 'Latvia')]
[('obj', 'http://www.wikidata.org/entity/Q214'), ('name', 'Slovakia')]
[('obj', 'http://www.wikidata.org/entity/Q215'), ('name', 'Slovenia')]
[('obj', 'http://www.wikidata.org/entity/Q228'), ('name', 'Andorra')]
[('obj', 'http://www.wikidata.org/entity/Q229'), ('name', 'Cyprus')]
[('obj', 'http://www.wikidata.org/entity/Q233'), ('name', 'Malta')]
[('obj', 'http://www.wikidata.org/entity/Q235'), ('name', 'Monaco')]
[('obj', 'http://www.wikidata.org/entity/Q237'), ('name', 'Vatican City')]
[('obj', 'http://www.wikidata.org/entity/Q238'), ('name', 'San Marino')]
[('obj', 'http://www.wikidata.org/entity/Q27'), ('name', 'Ireland')]
[('obj', 'http://www.wikidata.org/entity/Q29'), ('name', 'Spain')]
[('obj', 'ht

In [10]:
obj = [{"uri":r[0][1],"name":r[1][1]} for r in x]
evaluation.add_result(evaluation.get_index_workflow(pt),"2", evaluation.TYPE_SET ,"uri", obj)

The index of this workflow is: 1_1
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow1_1.json
JSON object updated


## Task 3

In [11]:
# use the property has part to object of euro-country-obj
queryString = """
SELECT DISTINCT ?x ?name
WHERE { 
    wd:Q4916 wdt:P17 ?obj.
    ?obj wdt:P361 ?x.
    ?x sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q1065'), ('name', 'United Nations')]
[('x', 'http://www.wikidata.org/entity/Q13116'), ('name', 'Benelux')]
[('x', 'http://www.wikidata.org/entity/Q215669'), ('name', 'Allies of the First World War')]
[('x', 'http://www.wikidata.org/entity/Q27293'), ('name', 'Western Asia')]
[('x', 'http://www.wikidata.org/entity/Q27468'), ('name', 'Eastern Europe')]
[('x', 'http://www.wikidata.org/entity/Q27496'), ('name', 'Western Europe')]
[('x', 'http://www.wikidata.org/entity/Q27509'), ('name', 'Central Europe')]
[('x', 'http://www.wikidata.org/entity/Q29999'), ('name', 'Kingdom of the Netherlands')]
[('x', 'http://www.wikidata.org/entity/Q39731'), ('name', 'Baltic states')]
[('x', 'http://www.wikidata.org/entity/Q458'), ('name', 'European Union')]
[('x', 'http://www.wikidata.org/entity/Q46'), ('name', 'Europe')]
[('x', 'http://www.wikidata.org/entity/Q476033'), ('name', 'Low Countries')]
[('x', 'http://www.wikidata.org/entity/Q50807777'), ('name', 'So

In [12]:
# property of European Union
queryString = """
SELECT DISTINCT ?prop ?name
WHERE { 
    wd:Q458 ?prop ?x.
    #FILTER(?curr != wd:Q4916).
    #FILTER NOT EXISTS{wd:Q4916 wdt:P17 ?obj.}
    ?prop sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P1051'), ('name', 'PSH ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P1081'), ('name', 'Human Development Index')]
[('prop', 'http://www.wikidata.org/prop/direct/P1082'), ('name', 'population')]
[('prop', 'http://www.wikidata.org/prop/direct/P112'), ('name', 'founded by')]
[('prop', 'http://www.wikidata.org/prop/direct/P1125'), ('name', 'Gini coefficient')]
[('prop', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal")]
[('prop', 'http://www.wikidata.org/prop/direct/P1245'), ('name', 'OmegaWiki Defined Meaning')]
[('prop', 'http://www.wikidata.org/prop/direct/P1273'), ('name', 'CANTIC ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P1280'), ('name', 'CONOR.SI ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('prop', 'http://www.wik

In [13]:
# looking for the part of the EU
queryString = """
SELECT DISTINCT ?x ?name
WHERE { 
    wd:Q458 wdt:P527 ?x.
    ?x sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q142'), ('name', 'France')]
[('x', 'http://www.wikidata.org/entity/Q183'), ('name', 'Germany')]
[('x', 'http://www.wikidata.org/entity/Q191'), ('name', 'Estonia')]
[('x', 'http://www.wikidata.org/entity/Q211'), ('name', 'Latvia')]
[('x', 'http://www.wikidata.org/entity/Q213'), ('name', 'Czech Republic')]
[('x', 'http://www.wikidata.org/entity/Q214'), ('name', 'Slovakia')]
[('x', 'http://www.wikidata.org/entity/Q215'), ('name', 'Slovenia')]
[('x', 'http://www.wikidata.org/entity/Q218'), ('name', 'Romania')]
[('x', 'http://www.wikidata.org/entity/Q219'), ('name', 'Bulgaria')]
[('x', 'http://www.wikidata.org/entity/Q224'), ('name', 'Croatia')]
[('x', 'http://www.wikidata.org/entity/Q229'), ('name', 'Cyprus')]
[('x', 'http://www.wikidata.org/entity/Q233'), ('name', 'Malta')]
[('x', 'http://www.wikidata.org/entity/Q28'), ('name', 'Hungary')]
[('x', 'http://www.wikidata.org/entity/Q29'), ('name', 'Spain')]
[('x', 'http://www.wikidata.org/entity/

In [14]:
# find countries which belong to EU but they don't uses Euro as currency
queryString = """
SELECT DISTINCT ?x ?name ?curr ?cName
WHERE { 
    wd:Q458 wdt:P527 ?x.
    ?x wdt:P38 ?curr.
    FILTER(?curr != wd:Q4916).
    ?x sc:name ?name.
    ?curr sc:name ?cName.
}
"""

print("Results")
x=run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q213'), ('name', 'Czech Republic'), ('curr', 'http://www.wikidata.org/entity/Q131016'), ('cName', 'Czech koruna')]
[('x', 'http://www.wikidata.org/entity/Q218'), ('name', 'Romania'), ('curr', 'http://www.wikidata.org/entity/Q131645'), ('cName', 'Romanian leu')]
[('x', 'http://www.wikidata.org/entity/Q219'), ('name', 'Bulgaria'), ('curr', 'http://www.wikidata.org/entity/Q172540'), ('cName', 'Bulgarian lev')]
[('x', 'http://www.wikidata.org/entity/Q224'), ('name', 'Croatia'), ('curr', 'http://www.wikidata.org/entity/Q26360'), ('cName', 'Croatian kuna')]
[('x', 'http://www.wikidata.org/entity/Q28'), ('name', 'Hungary'), ('curr', 'http://www.wikidata.org/entity/Q47190'), ('cName', 'forint')]
[('x', 'http://www.wikidata.org/entity/Q34'), ('name', 'Sweden'), ('curr', 'http://www.wikidata.org/entity/Q122922'), ('cName', 'Swedish krona')]
[('x', 'http://www.wikidata.org/entity/Q35'), ('name', 'Denmark'), ('curr', 'http://www.wikidata.org/entity/Q2

In [15]:
# URI associated to another URI
objs = []
for i in x:
    f_uri = i[0][1]
    f_name = i[1][1]
    c_uri = i[2][1]
    c_name = i[3][1]
    obj = {}
    obj["refers_to"] = f_uri
    obj["refers_to_name"] = f_name
    obj["check"] = ["uri","name"]
    obj["uri"]= c_uri
    obj["name"]= c_name
    objs.append(obj)

evaluation.add_result(evaluation.get_index_workflow(pt),"3", evaluation.TYPE_REFERRED ,["uri","name"], objs)

The index of this workflow is: 1_1
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow1_1.json
JSON object updated


## Task 4

In [16]:
# from one of the first query use P1365 replaces
queryString = """
SELECT DISTINCT ?x ?name
WHERE { 
    wd:Q4916 wdt:P1365 ?x.
    ?x sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q270642'), ('name', 'Irish pound')]
[('x', 'http://www.wikidata.org/entity/Q788472'), ('name', 'Dutch guilder')]
[('x', 'http://www.wikidata.org/entity/Q1546443'), ('name', 'Greek drachma')]
[('x', 'http://www.wikidata.org/entity/Q203354'), ('name', 'Finnish markka')]
[('x', 'http://www.wikidata.org/entity/Q204992'), ('name', 'Italian lira')]
[('x', 'http://www.wikidata.org/entity/Q468474'), ('name', 'Portuguese escudo')]
[('x', 'http://www.wikidata.org/entity/Q184172'), ('name', 'French franc')]
[('x', 'http://www.wikidata.org/entity/Q189097'), ('name', 'peseta')]
[('x', 'http://www.wikidata.org/entity/Q16068'), ('name', 'Deutsche Mark')]
[('x', 'http://www.wikidata.org/entity/Q174467'), ('name', 'Maltese lira')]
[('x', 'http://www.wikidata.org/entity/Q133011'), ('name', 'Latvian lats')]
[('x', 'http://www.wikidata.org/entity/Q183354'), ('name', 'Vatican lira')]
[('x', 'http://www.wikidata.org/entity/Q129983'), ('name', 'Estonian kroon')]

In [18]:
# from one of the first query use P1365 replaces
queryString = """
SELECT DISTINCT ?country ?cName ?x ?name 
WHERE { 
    wd:Q4916 wdt:P1365 ?x.
    ?country wdt:P38 wd:Q4916.
    ?x wdt:P17 ?country.
    ?x sc:name ?name.
    ?country sc:name ?cName.
}
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q27'), ('cName', 'Ireland'), ('x', 'http://www.wikidata.org/entity/Q270642'), ('name', 'Irish pound')]
[('country', 'http://www.wikidata.org/entity/Q29999'), ('cName', 'Kingdom of the Netherlands'), ('x', 'http://www.wikidata.org/entity/Q788472'), ('name', 'Dutch guilder')]
[('country', 'http://www.wikidata.org/entity/Q41'), ('cName', 'Greece'), ('x', 'http://www.wikidata.org/entity/Q1546443'), ('name', 'Greek drachma')]
[('country', 'http://www.wikidata.org/entity/Q33'), ('cName', 'Finland'), ('x', 'http://www.wikidata.org/entity/Q203354'), ('name', 'Finnish markka')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('cName', 'Italy'), ('x', 'http://www.wikidata.org/entity/Q204992'), ('name', 'Italian lira')]
[('country', 'http://www.wikidata.org/entity/Q45'), ('cName', 'Portugal'), ('x', 'http://www.wikidata.org/entity/Q468474'), ('name', 'Portuguese escudo')]
[('country', 'http://www.wikidata.org/entity/Q235'), ('cName', 'Monac

In [19]:
# URI associated to another URI
objs = []
for i in x:
    f_uri = i[0][1]
    f_name = i[1][1]
    c_uri = i[2][1]
    c_name = i[3][1]
    obj = {}
    obj["refers_to"] = f_uri
    obj["refers_to_name"] = f_name
    obj["check"] = ["uri","name"]
    obj["uri"]= c_uri
    obj["name"]= c_name
    objs.append(obj)

evaluation.add_result(evaluation.get_index_workflow(pt),"4", evaluation.TYPE_REFERRED ,["uri","name"], objs)

The index of this workflow is: 1_1
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow1_1.json
JSON object updated


## Task 5

In [20]:
# find dollars
queryString = """
SELECT DISTINCT ?x ?name 
WHERE { 
    ?x wdt:P31 wd:Q8142.
    #?country wdt:P38 ?x.
    ?x sc:name ?name.
    FILTER(REGEX(?name,"dollar")).
}
"""

print("Results")
x=run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q242890'), ('name', 'Trinidad and Tobago dollar')]
[('x', 'http://www.wikidata.org/entity/Q942092'), ('name', 'British West Indies dollar')]
[('x', 'http://www.wikidata.org/entity/Q1104069'), ('name', 'Canadian dollar')]
[('x', 'http://www.wikidata.org/entity/Q259502'), ('name', 'Australian dollar')]
[('x', 'http://www.wikidata.org/entity/Q4597'), ('name', 'Solomon Islands dollar')]
[('x', 'http://www.wikidata.org/entity/Q4917'), ('name', 'United States dollar')]
[('x', 'http://www.wikidata.org/entity/Q2411666'), ('name', 'Newfoundland dollar')]
[('x', 'http://www.wikidata.org/entity/Q209792'), ('name', 'Jamaican dollar')]
[('x', 'http://www.wikidata.org/entity/Q15039722'), ('name', 'spanish dollar')]
[('x', 'http://www.wikidata.org/entity/Q31015'), ('name', 'Hong Kong dollar')]
[('x', 'http://www.wikidata.org/entity/Q261447'), ('name', 'Straits dollar')]
[('x', 'http://www.wikidata.org/entity/Q4592'), ('name', 'Fijian dollar')]
[('x', 'ht

In [21]:
# find some dollar classes
queryString = """
SELECT DISTINCT ?y ?n 
WHERE { 
    ?x wdt:P31 wd:Q8142.
    ?x wdt:P31 ?y.
    ?x sc:name ?name.
    FILTER(REGEX(?name,"dollar")).
    ?y sc:name ?n.
}
"""

print("Results")
x=run_query(queryString)

Results
[('y', 'http://www.wikidata.org/entity/Q14083'), ('n', 'dollar')]
[('y', 'http://www.wikidata.org/entity/Q8142'), ('n', 'currency')]
[('y', 'http://www.wikidata.org/entity/Q28795800'), ('n', 'decimal currency')]
[('y', 'http://www.wikidata.org/entity/Q25530138'), ('n', 'yuan')]
[('y', 'http://www.wikidata.org/entity/Q1643989'), ('n', 'legal tender')]
[('y', 'http://www.wikidata.org/entity/Q756202'), ('n', 'reserve currency')]
6


In [22]:
# list instances of dollar
queryString = """
SELECT DISTINCT ?x ?name 
WHERE { 
    ?x wdt:P31 wd:Q14083.
    ?x sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q242890'), ('name', 'Trinidad and Tobago dollar')]
[('x', 'http://www.wikidata.org/entity/Q942092'), ('name', 'British West Indies dollar')]
[('x', 'http://www.wikidata.org/entity/Q1104069'), ('name', 'Canadian dollar')]
[('x', 'http://www.wikidata.org/entity/Q259502'), ('name', 'Australian dollar')]
[('x', 'http://www.wikidata.org/entity/Q4597'), ('name', 'Solomon Islands dollar')]
[('x', 'http://www.wikidata.org/entity/Q4917'), ('name', 'United States dollar')]
[('x', 'http://www.wikidata.org/entity/Q2411666'), ('name', 'Newfoundland dollar')]
[('x', 'http://www.wikidata.org/entity/Q209792'), ('name', 'Jamaican dollar')]
[('x', 'http://www.wikidata.org/entity/Q392858'), ('name', 'Rhodesian dollar')]
[('x', 'http://www.wikidata.org/entity/Q31015'), ('name', 'Hong Kong dollar')]
[('x', 'http://www.wikidata.org/entity/Q4592'), ('name', 'Fijian dollar')]
[('x', 'http://www.wikidata.org/entity/Q4817670'), ('name', 'Coins and postage stamps of

In [23]:
obj = [{"uri":r[0][1],"name":r[1][1]} for r in x]
evaluation.add_result(evaluation.get_index_workflow(pt),"5", evaluation.TYPE_SET ,"uri", obj)

The index of this workflow is: 1_1
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow1_1.json
JSON object updated


## Task 6

In [62]:
# find currencies used in more than one countries
queryString = """
SELECT DISTINCT ?x ?name (COUNT(DISTINCT ?country) AS ?count)
WHERE { 
    ?x wdt:P31 wd:Q8142.
    ?country wdt:P38 ?x;
        wdt:P31/wdt:P279* wd:Q6256.
    ?x sc:name ?name.
}
GROUP BY ?x ?name
HAVING (COUNT(DISTINCT ?country)>1)
ORDER BY DESC (?count)
"""

print("Results")
x=run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q4916'), ('name', 'euro'), ('count', '25')]
[('x', 'http://www.wikidata.org/entity/Q4917'), ('name', 'United States dollar'), ('count', '12')]
[('x', 'http://www.wikidata.org/entity/Q861690'), ('name', 'West African CFA franc'), ('count', '8')]
[('x', 'http://www.wikidata.org/entity/Q26365'), ('name', 'Eastern Caribbean dollar'), ('count', '6')]
[('x', 'http://www.wikidata.org/entity/Q847739'), ('name', 'Central African CFA franc'), ('count', '6')]
[('x', 'http://www.wikidata.org/entity/Q199462'), ('name', 'Egyptian pound'), ('count', '4')]
[('x', 'http://www.wikidata.org/entity/Q259502'), ('name', 'Australian dollar'), ('count', '4')]
[('x', 'http://www.wikidata.org/entity/Q25224'), ('name', 'pound sterling'), ('count', '3')]
[('x', 'http://www.wikidata.org/entity/Q181907'), ('name', 'South African rand'), ('count', '3')]
[('x', 'http://www.wikidata.org/entity/Q25417'), ('name', 'Danish krone'), ('count', '3')]
[('x', 'http://www.wikidata

## Task 7

In [76]:
# from the first query I remember the property population P1082
queryString = """
SELECT DISTINCT ?x ?name (SUM(?maxP) AS ?count)
WHERE { 
    ## sub query
    {
        SELECT ?country MAX(?population) AS ?maxP WHERE{
            ?country wdt:P38 ?x;
                wdt:P31/wdt:P279* wd:Q6256.
            OPTIONAL{?country wdt:P1082 ?population.}
        }
        GROUP BY ?country
    }
    ?x wdt:P31 wd:Q8142.
    ?country wdt:P38 ?x;
        wdt:P31/wdt:P279* wd:Q6256.
    OPTIONAL{
        ?country wdt:P1082 ?population.
        FILTER(?population = ?maxP).
    }
    ?x sc:name ?name.
}
GROUP BY ?x ?name
HAVING (COUNT(DISTINCT ?country)>1)
ORDER BY DESC (?count)
"""

print("Results")
x=run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q25224'), ('name', 'pound sterling'), ('count', '6106448730')]
[('x', 'http://www.wikidata.org/entity/Q4916'), ('name', 'euro'), ('count', '1613516682')]
[('x', 'http://www.wikidata.org/entity/Q4917'), ('name', 'United States dollar'), ('count', '1553254316')]
[('x', 'http://www.wikidata.org/entity/Q8146'), ('name', 'Japanese yen'), ('count', '954147390')]
[('x', 'http://www.wikidata.org/entity/Q199462'), ('name', 'Egyptian pound'), ('count', '947988270')]
[('x', 'http://www.wikidata.org/entity/Q206243'), ('name', 'birr'), ('count', '629744628')]
[('x', 'http://www.wikidata.org/entity/Q861690'), ('name', 'West African CFA franc'), ('count', '599518780')]
[('x', 'http://www.wikidata.org/entity/Q199471'), ('name', 'Afghan afghani'), ('count', '358234802')]
[('x', 'http://www.wikidata.org/entity/Q188608'), ('name', 'Iranian rial'), ('count', '319864920')]
[('x', 'http://www.wikidata.org/entity/Q181907'), ('name', 'South African rand'), ('coun

In [70]:
# from the first query I remember the property population P1082
queryString = """
SELECT DISTINCT ?x ?name (SUM(?population) AS ?count)
WHERE { 
    ?x wdt:P31 wd:Q8142.
    ?country wdt:P38 ?x;
        wdt:P31/wdt:P279* wd:Q6256;
        wdt:P1082 ?population.
    ?x sc:name ?name.
}
GROUP BY ?x ?name
HAVING (COUNT(DISTINCT ?country)>1)
ORDER BY DESC (?count)
"""

print("Results")
x=run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q25224'), ('name', 'pound sterling'), ('count', '6106448730')]
[('x', 'http://www.wikidata.org/entity/Q4916'), ('name', 'euro'), ('count', '1652751098')]
[('x', 'http://www.wikidata.org/entity/Q4917'), ('name', 'United States dollar'), ('count', '1553365694')]
[('x', 'http://www.wikidata.org/entity/Q199462'), ('name', 'Egyptian pound'), ('count', '973490270')]
[('x', 'http://www.wikidata.org/entity/Q8146'), ('name', 'Japanese yen'), ('count', '954147390')]
[('x', 'http://www.wikidata.org/entity/Q172872'), ('name', 'Turkish lira'), ('count', '836770872')]
[('x', 'http://www.wikidata.org/entity/Q861690'), ('name', 'West African CFA franc'), ('count', '606963912')]
[('x', 'http://www.wikidata.org/entity/Q199471'), ('name', 'Afghan afghani'), ('count', '358234802')]
[('x', 'http://www.wikidata.org/entity/Q41044'), ('name', 'Russian ruble'), ('count', '293715808')]
[('x', 'http://www.wikidata.org/entity/Q181907'), ('name', 'South African rand')